# Interpolating file sequences

For cases in which interpolation is needed, one must interpolate a series of files. Here we show how that can be done easily.

#### Import general modules

In [1]:
# Import required modules
from mpi4py import MPI #equivalent to the use of MPI_init() in C
import matplotlib.pyplot as plt
import numpy as np

# Get mpi info
comm = MPI.COMM_WORLD

#### Import modules from pynektools

## Initialize the field series interpolator

In [2]:
query_points_fname = "points.hdf5"
data_path = "../data/sem_data/instantaneous/cylinder_rbc_nelv_600/"
sem_mesh_fname = f"{data_path}field0.f00801"

# Probes optional inputs
interpolated_fields_output_fname = "interpolated_fields.hdf5"
dtype = np.single
write_coords = True
point_interpolator_type='multiple_point_legendre_numpy'
max_pts=256
find_points_comm_pattern='point_to_point'
elem_percent_expansion = 0.01
global_tree_type = "rank_bbox"
global_tree_nbins = 1024,
use_autograd = False
find_points_tol = np.finfo(np.double).eps * 10,
find_points_max_iter = 50

In [ ]:
from pynektools.io.ppymech.neksuite import pynekread
from pynektools.datatypes.msh import Mesh
from pynektools.datatypes.field import FieldRegistry
from pynektools.interpolation.probes import Probes

probes = Probes(comm,
        output_fname = interpolated_fields_output_fname,
        probes = query_points_fname,
        msh = sem_mesh_fname,
        write_coords  = write_coords,
        point_interpolator_type = point_interpolator_type,
        max_pts = max_pts,
        find_points_comm_pattern = find_points_comm_pattern ,
        elem_percent_expansion = elem_percent_expansion,
        global_tree_type = global_tree_type,
        global_tree_nbins = global_tree_nbins,
        use_autograd = use_autograd,
        find_points_tol = find_points_tol,
        find_points_max_iter = find_points_max_iter,
    )


In [ ]:
field_interpolation_dictionary = {}

field_interpolation_dictionary['input_type'] = "file_sequence" # or file_index
field_interpolation_dictionary['file_sequence'] = [f"{data_path}field0.f{str(i).zfill(5)}" for i in range(801, 850)]
field_interpolation_dictionary['fields_to_interpolate'] = ["all"]

fld = FieldRegistry(comm)

for fname in field_interpolation_dictionary['file_sequence']:

    # Read the field in the sequence    
    pynekread(fname, comm, data_dtype=dtype, fld=fld)

    # Prepare a list with the inputs
    if field_interpolation_dictionary['fields_to_interpolate'] == ["all"]:
        field_list = [fld.registry[key] for key in fld.registry.keys()] 
        field_names = [key for key in fld.registry.keys()]
    else:
        field_list = [fld.registry[key] for key in field_interpolation_dictionary['fields_to_interpolate']]
        field_names = field_interpolation_dictionary['fields_to_interpolate']
        
    # Interpolate the fields
    probes.interpolate_from_field_list(fld.t, field_list, comm, write_data=True, field_names=field_names)

    # Clear the fields
    fld.clear()
